In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, concatenate
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import os
import tensorflow as tf

# ========== STEP 1: Preprocess Spectral Data ==========
spectral_data = df.iloc[:, 3:21].values
scaler = StandardScaler()
spectral_data_scaled = scaler.fit_transform(spectral_data)

# ========== STEP 2: Load and Resize Images ==========
IMG_SIZE = (128, 128)
image_data = []

for i, row in df.iterrows():
    image_path = os.path.join(row['folder'].strip('/'), row['image_name'])
    try:
        img = load_img(image_path, target_size=IMG_SIZE)
        img_array = img_to_array(img) / 255.0  # Normalize
        image_data.append(img_array)
    except Exception as e:
        print(f"Error loading {image_path}: {e}")
        image_data.append(np.zeros((*IMG_SIZE, 3)))  # placeholder if not found

image_data = np.array(image_data)

# ========== STEP 3: Encode Class Labels ==========
labels = LabelEncoder().fit_transform(df['class'].values)

# ========== STEP 4: Split ==========
X_img_train, X_img_test, X_spec_train, X_spec_test, y_train, y_test = train_test_split(
    image_data, spectral_data_scaled, labels, test_size=0.2, random_state=42)

# ========== STEP 5: CNN for Image Branch ==========
img_input = Input(shape=(*IMG_SIZE, 3))
x = Conv2D(32, (3,3), activation='relu')(img_input)
x = MaxPooling2D()(x)
x = Conv2D(64, (3,3), activation='relu')(x)
x = MaxPooling2D()(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

# ========== STEP 6: Dense for Spectrometer Branch ==========
spec_input = Input(shape=(18,))
y = Dense(64, activation='relu')(spec_input)
y = Dropout(0.3)(y)
y = Dense(32, activation='relu')(y)

# ========== STEP 7: Combine Both ==========
combined = concatenate([x, y])
z = Dense(64, activation='relu')(combined)
z = Dropout(0.3)(z)
z = Dense(1, activation='sigmoid')(z)

# ========== STEP 8: Final Model ==========
model = Model(inputs=[img_input, spec_input], outputs=z)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ========== STEP 9: Train ==========
model.fit([X_img_train, X_spec_train], y_train, 
          validation_data=([X_img_test, X_spec_test], y_test),
          epochs=30, batch_size=8)
